In [12]:
from bs4 import BeautifulSoup
import requests
import os
import lxml
import re

In [13]:
__author__ = "Mike Zhong"

## This notebook will walk through the steps I took to scrape a single table from an html page, the methods used here will be converted into a function and looped over all 16 weeks of the NFL season to generate 16 .csv files. They will then be opened and manipulated using R

In [14]:
print(os.getcwd())
wd = os.getcwd()

C:\Users\mike.zhong\Dropbox\dev\fantasy_analytics


In [15]:
my_url = "http://www.footballdb.com/fantasy-football/index.html?pos=QB%2CRB%2CWR%2CTE&yr=2016&wk=1&rules=1"

html_page = requests.get(my_url)

if html_page.status_code != 200:
    print("Non-200 response code returned, exiting...")
    exit(1)

print(type(html_page))
print(html_page)
print(html_page.text[:400])

with open(wd + "/html/sample_html.html", 'w+') as fp:
    fp.write(html_page.text)

<class 'requests.models.Response'>
<Response [200]>
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="viewport" content="initial-scale=1.0, width=device-width" />
<title>2016 Fantasy Football Statistics | The Football Database</title>
<meta name="description" content="View week 1 QB,RB,WR,TE fantasy football stats and statistics for the 2016 NFL season. Included are 


In [16]:
soup = BeautifulSoup(html_page.text, 'lxml')

print(type(soup))

all_tables = soup.find_all("table")
print(len(all_tables), type(all_tables))

<class 'bs4.BeautifulSoup'>
7 <class 'bs4.element.ResultSet'>


In [17]:
table = all_tables[0]
trs = table.find_all("tr")
print(len(trs), type(trs))

tr = trs[1]
ths = tr.find_all("th")
print(len(ths), type(ths))

102 <class 'bs4.element.ResultSet'>
19 <class 'bs4.element.ResultSet'>


In [18]:
headers = trs[1].text.strip().split('\n')
ncols = len(headers)
print(len(headers), type(headers))

tds = trs[2].find_all('td')
print(len(tds))

19 <class 'list'>
19


In [19]:
# modify headers to accomodate my DF structure
headers
headers.remove('Game')
headers.insert(1, 'Team')
headers.insert(2, 'Opponent')
headers.append('Week')
headers

['Player',
 'Team',
 'Opponent',
 'Pts*',
 'Att',
 'Cmp',
 'Yds',
 'TD',
 'Int',
 '2Pt',
 'Att',
 'Yds',
 'TD',
 '2Pt',
 'Rec',
 'Yds',
 'TD',
 '2Pt',
 'FL',
 'TD',
 'Week']

In [20]:
import pandas as pd

rows = []
for tr in trs[2:]:
    
    row = []
    
    for td in tr.find_all('td'):
        
        if len(td) > 0:    
            text = td.text.replace("\xa0", " ").replace('.', '')
            
            # get team of player and opposing team, and home/away
            bold = td.find('b')
            if bold:
                row.append(bold.text)
                m = re.match(r"^(.+)@(.+)$", text)
                if m and text.startswith(bold.text):
                    row.append(m.groups()[1])
                else:
                    row.append('@' + m.groups()[0])
                continue
                
            
            # parse player name from awful text
            m = re.match(r"^(.+) (.+)", text)
            if m:
                name = m.groups()[0][:-1]
                name
                row.append(name)
                continue
                
            row.append(text)
    row.append(1)
    rows.append(row)

rows
my_table = pd.DataFrame(rows, columns=headers)

#print(my_table.size)
#print(my_table.shape)
some_row = my_table[1:13]

print(some_row, type(some_row))

                Player Team Opponent  Pts* Att Cmp  Yds TD Int 2Pt ...   Yds  \
1           Drew Brees   NO     @OAK  3900  42  28  423  4   0   0 ...     5   
2       Jameis Winston   TB      ATL  3300  32  23  281  4   1   0 ...     3   
3     Matthew Stafford  DET      IND  3100  39  31  340  3   0   0 ...     5   
4           Alex Smith   KC      @SD  3100  48  34  363  2   1   0 ...    15   
5    DeAngelo Williams  PIT      WAS  2800   0   0    0  0   0   0 ...   143   
6            Matt Ryan  ATL      @TB  2800  39  27  334  2   0   1 ...    10   
7   Ben Roethlisberger  PIT      WAS  2800  37  27  300  3   1   0 ...    -2   
8        Brandin Cooks   NO     @OAK  2700   0   0    0  0   0   0 ...    11   
9        Aaron Rodgers   GB      JAX  2600  34  20  199  2   0   0 ...    16   
10        Spencer Ware   KC      @SD  2500   0   0    0  0   0   0 ...    70   
11         CJ Anderson  DEN     @CAR  2500   0   0    0  0   0   0 ...    92   
12         Eli Manning  NYG      DAL  24

In [21]:
my_table.to_csv(wd + "/csv/sample_table.csv", index=False)

In [22]:
# This is okay but we want to clean up the 'Player' column, use regex to pull out firs tand last names
some_name = 'Drew BreesD Brees'
m = re.match(r"^(.+) (.+)", some_name)
name = m.groups()[0][:-1]
name
# This is working so I'll plug into the function above and re-run to test it out

'Drew Brees'